### Import Modules

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
from folium.features import ColorLine
import rasterstats
import json
import geopy
import branca.colormap as cm
import math

# Initialize map

In [2]:
# Determine Map center
UW = [47.655548, -122.303200]

#Create the map
m = folium.Map(location = UW, zoom_start = 12)
m

# Read shp file, extract route45 shp for next step to split into linestrings

In [71]:
fp = '../data/Transit_Routes_for_King_County_Metro__transitroute_line.shp'
data = gpd.read_file(fp)
dat = data[data['IN_SERVICE']=='Y']
da = dat[data['ROUTE_NUM'] == 45]
da

,OBJECTID,CHANGE_NUM,MINOR_CHAN,CURRENT_NE,IN_SERVICE,ROUTE_ID,LOCAL_EXPR,ROUTE_NUM,SHAPE_Leng,geometry
140,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,(LINESTRING (-122.3075605352953 47.65053260749...


# break down multi linestring to single linestring, save it as lines.shp

In [72]:
import fiona
from shapely.geometry import  MultiLineString, mapping, shape
with fiona.open('da.shp') as source:
    # create the new shapefile
    with fiona.open('lines.shp','w', driver='ESRI Shapefile',crs=source.crs,schema=source.schema) as ouput:
        # iterate the features of the original shapefile
        for elem in source:
               # iterate the list of geometries in one element (split the MultiLineString)
            for line in shape(elem['geometry']): 
                # write the line to the new shapefile
                ouput.write({'geometry':mapping(line),'properties':elem['properties']})

In [76]:
gpd.read_file('lines.shp')

,OBJECTID,CHANGE_NUM,MINOR_CHAN,CURRENT_NE,IN_SERVICE,ROUTE_ID,LOCAL_EXPR,ROUTE_NUM,SHAPE_Leng,geometry
0,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3075605352953 47.650532607492...
1,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3075605352953 47.650532607492...
2,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3107275034277 47.652094806596...
3,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3115149015316 47.652484919657...
4,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3115149015316 47.652484919657...
5,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3124628890345 47.652984815362...
6,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3132232503292 47.653393638881...
7,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3132232503292 47.653393638881...
8,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3132461109814 47.656834269145...
9,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3132070900588 47.658314904223...


# Get grade

In [121]:
sh= rasterstats.point_query(route45, '../data/sea_north.tif')
elev = pd.DataFrame(sh).T
# elev
elev[0].values
grade_elev = np.diff(elev[0].values)
# grade_elev.shape
# grade_elev = np.insert(grade_elev,0, 0)
# grade_elev

In [57]:
route45

,SHAPE_Leng,ROUTE_NUM,Id,OBJECTID,geometry
2,0.0,45,0,0,LINESTRING Z (-122.3042444001078 47.6504264662...


# get points, calculate distance between two points in line segment

In [3]:
sh= rasterstats.zonal_stats('lines.shp', '../data/sea_north.tif')
SH = pd.DataFrame(sh)
SH

,count,max,mean,min
0,811,99.969002,70.771164,60.937389
1,310,119.307007,67.954095,58.150391
2,78,102.046722,82.531062,60.512062
3,78,100.254448,82.134315,60.512062
4,93,100.451881,75.590343,68.369812
5,75,89.578796,72.041016,69.613693
6,75,89.578796,72.034206,69.613693
7,342,140.569778,98.127821,68.442673
8,146,184.838211,147.069657,128.673935
9,146,184.838211,147.066553,128.673935


In [58]:
shapefile.geometry.values

array([<shapely.geometry.linestring.LineString object at 0x000000D47E5BBEF0>,
      dtype=object)

In [74]:
from shapely.geometry import mapping
shapefile = gpd.read_file("../data/six_routes.shp")
route45 = shapefile[shapefile['ROUTE_NUM'] == 45]
# extract the geometry in GeoJSON format
geoms = route45.geometry.values # list of shapely geometries
geoms = [mapping(geoms[0])]
r = geoms[0]['coordinates']

In [75]:
linestring = []
for i in range(len(r)):
    linestring.append(geoms[0]['coordinates'][i][:2])
len(linestring)

208

In [483]:
li = []
for i in range(len(geoms[0]['coordinates'])):
    li.append(len(geoms[0]['coordinates'][i]))
    
group = np.array(li)

In [114]:
linestringDF.loc[0][1]

(-122.30424440010776, 47.65042646627143)

In [115]:
from shapely.geometry import LineString
def make_lines(gdf, df_out, i, geometry = 'geometry'):    
    geom0 = linestringDF.loc[i][1]
    geom1 = linestringDF.loc[i+1][1]
    start, end = [geom0, geom1]
    line = LineString([start, end])
    
    # Create a DataFrame to hold record
    data = {'grade':i,
            'geometry':[line]}
    df_line = pd.DataFrame(data, columns = ['grade', 'geometry'])
    
    # Add record DataFrame of compiled records
    df_out = pd.concat([df_out, df_line])
    return df_out

# convert list to DF

In [102]:
linestringDF = pd.DataFrame(grade_elev, columns = ['grade'])
linestringDF['coord'] = linestring
# ({'coor':linestring},columns=['id','coor'])
# for i in range(len(linestringDF)):
#     linestringDF.loc[i][0] = i
# linestringDF['grade'] = grade_elev
linestringDF

,grade,coord
0,0.000000,"(-122.30424440010776, 47.65042646627143)"
1,0.196298,"(-122.30424998379348, 47.65041485164514)"
2,0.123942,"(-122.30427784808448, 47.650359884437)"
3,2.343023,"(-122.30431345405354, 47.65028282713853)"
4,-1.070188,"(-122.30435529243597, 47.650173880453444)"
5,0.191170,"(-122.30439282260569, 47.65007615230654)"
6,0.809978,"(-122.30447059928024, 47.64987059548755)"
7,2.024472,"(-122.30453122432124, 47.649646888555935)"
8,2.816997,"(-122.30458069194579, 47.64938740558821)"
9,3.539533,"(-122.3046732123107, 47.64931574177743)"


In [566]:
geoms[0]['coordinates'][0][2]

(-122.30722583308294, 47.65059260283979)

In [110]:
len(linestringDF)

208

In [123]:
df = pd.DataFrame(columns = ['grade', 'geometry'])
x = 0 
while x<len(linestringDF) -1:
    df = make_lines(linestringDF,df,x)
    x = x+1
df['grade'] = grade_elev
df

,grade,geometry
0,0.196298,LINESTRING (-122.3042444001078 47.650426466271...
0,0.123942,LINESTRING (-122.3042499837935 47.650414851645...
0,2.343023,LINESTRING (-122.3042778480845 47.650359884437...
0,-1.070188,LINESTRING (-122.3043134540535 47.650282827138...
0,0.191170,LINESTRING (-122.304355292436 47.6501738804534...
0,0.809978,LINESTRING (-122.3043928226057 47.650076152306...
0,2.024472,LINESTRING (-122.3044705992802 47.649870595487...
0,2.816997,LINESTRING (-122.3045312243212 47.649646888555...
0,3.539533,LINESTRING (-122.3045806919458 47.649387405588...
0,-6.040008,LINESTRING (-122.3046732123107 47.649315741777...


In [559]:
end = []
j = 2
for i in range(j):
    t = len(geoms[0]['coordinates'][i])
    end.append(t)
    sum = np.sum(end)
end

[36, 5]

# DF - GDF

In [125]:
linestringGDF = gpd.GeoDataFrame(df)
linestringGDF

,grade,geometry
0,0.196298,LINESTRING (-122.3042444001078 47.650426466271...
0,0.123942,LINESTRING (-122.3042499837935 47.650414851645...
0,2.343023,LINESTRING (-122.3042778480845 47.650359884437...
0,-1.070188,LINESTRING (-122.3043134540535 47.650282827138...
0,0.191170,LINESTRING (-122.304355292436 47.6501738804534...
0,0.809978,LINESTRING (-122.3043928226057 47.650076152306...
0,2.024472,LINESTRING (-122.3044705992802 47.649870595487...
0,2.816997,LINESTRING (-122.3045312243212 47.649646888555...
0,3.539533,LINESTRING (-122.3045806919458 47.649387405588...
0,-6.040008,LINESTRING (-122.3046732123107 47.649315741777...


In [42]:
from geopy.distance import geodesic
geodesic([47.65086210845893,-122.30820251335712],[47.65102755761703,-122.30857526433199]).m

33.50517341257214

#  plot line segments

In [82]:
r = gpd.read_file('lines.shp')
R = r.to_json()
r45 = folium.GeoJson(R)
m.add_child(r45)
m

# below are pointless for now

# Geojson Map add max elevation (as elevation_max) for each line shape to GDF and a covert to GeoJson, then include in the style function

In [83]:
r['elevation_max'] = SH.T.max()

In [91]:
ele_min = SH.T.min()
ele_min

0       61.121241
1       60.484874
2       60.484874
3       60.484874
4       71.509512
5       69.619129
6       69.619129
7       68.599417
8      129.678151
9      129.678151
10     155.162508
11     155.162508
12     183.864970
13     177.683419
14     195.938721
15     195.938721
16     212.699334
17     248.266544
18     212.699334
19     223.137782
20     223.137782
21     248.266544
22     252.505608
23     223.137782
24     231.175765
25     252.505608
26     247.145972
27     231.175765
28     232.569888
29     225.174791
          ...    
105    338.823497
106    338.823497
107    338.823497
108    339.077881
109    339.077881
110    339.044061
111    338.966526
112    327.450368
113    327.450368
114    317.802427
115    334.016220
116    334.016220
117    321.197336
118    299.157711
119    299.157711
120    296.928320
121    301.968826
122    301.968826
123    308.334072
124    307.694467
125    307.694467
126    306.127784
127    306.127784
128    289.135245
129    276

In [98]:
r

,OBJECTID,CHANGE_NUM,MINOR_CHAN,CURRENT_NE,IN_SERVICE,ROUTE_ID,LOCAL_EXPR,ROUTE_NUM,SHAPE_Leng,geometry,elevation_max
0,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3075605352953 47.650532607492...,92.129986
1,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3075605352953 47.650532607492...,71.715242
2,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3107275034277 47.652094806596...,96.856224
3,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3115149015316 47.652484919657...,96.856224
4,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3115149015316 47.652484919657...,96.856224
5,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3124628890345 47.652984815362...,71.509512
6,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3132232503292 47.653393638881...,71.509512
7,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3132232503292 47.653393638881...,129.678151
8,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3132461109814 47.656834269145...,155.162508
9,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3132070900588 47.658314904223...,155.162508


## Try to output route45.json

In [126]:
R = linestringGDF.to_json()
R

'{"type": "FeatureCollection", "features": [{"id": "0", "type": "Feature", "properties": {"grade": 0.19629820559886468}, "geometry": {"type": "LineString", "coordinates": [[-122.30424440010776, 47.65042646627143], [-122.30424998379348, 47.65041485164514]]}}, {"id": "0", "type": "Feature", "properties": {"grade": 0.12394179633834312}, "geometry": {"type": "LineString", "coordinates": [[-122.30424998379348, 47.65041485164514], [-122.30427784808448, 47.650359884437]]}}, {"id": "0", "type": "Feature", "properties": {"grade": 2.3430230196024837}, "geometry": {"type": "LineString", "coordinates": [[-122.30427784808448, 47.650359884437], [-122.30431345405354, 47.65028282713853]]}}, {"id": "0", "type": "Feature", "properties": {"grade": -1.07018809713567}, "geometry": {"type": "LineString", "coordinates": [[-122.30431345405354, 47.65028282713853], [-122.30435529243597, 47.650173880453444]]}}, {"id": "0", "type": "Feature", "properties": {"grade": 0.19116992967065016}, "geometry": {"type": "Lin

In [297]:
with open('Route_45.json', 'w') as json_file:
    json.dump(R, json_file)

In [127]:
import branca.colormap as cm
import math

Seattle_coords = [47.655548, -122.303200]
# f2 = folium.Figure(height = 400)
m2 = folium.Map(location = Seattle_coords, zoom_start = 12)

max_elev = max(grade_elev)
linear = cm.linear.Paired_12.scale(0, max_elev )
gps_lyr = folium.GeoJson(R, style_function = lambda feature: {
                             'color': linear(feature['properties']['grade']),
                             'weight': 5})

gps_lyr.add_child

m2.add_child(linear)
m2.add_child(gps_lyr)
# m2.add_to(f2)

In [93]:
import branca.colormap as cm
import math

Seattle_coords = [47.655548, -122.303200]
# f2 = folium.Figure(height = 400)
m2 = folium.Map(location = Seattle_coords, zoom_start = 12)

max_elev = max(r['elevation_max'])
linear = cm.linear.RdBu_06.scale(ele_min.min(), max_elev )
gps_lyr = folium.GeoJson(R, style_function = lambda feature: {
                             'color': linear(feature['properties']['elevation_max']),
                             'weight': 5})

gps_lyr.add_child

m2.add_child(linear)
m2.add_child(gps_lyr)
# m2.add_to(f2)

In [ ]:
fp = '../data/Transit_Routes_for_King_County_Metro__transitroute_line.shp'
data = gpd.read_file(fp)
dat = data[data['IN_SERVICE']=='Y']
da = dat[data['ROUTE_NUM'] == 45]
da

In [445]:
r

,OBJECTID,CHANGE_NUM,MINOR_CHAN,CURRENT_NE,IN_SERVICE,ROUTE_ID,LOCAL_EXPR,ROUTE_NUM,SHAPE_Leng,geometry,elevation_max
0,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3075605352953 47.650532607492...,92.129986
1,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3075605352953 47.650532607492...,71.715242
2,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3107275034277 47.652094806596...,96.856224
3,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3115149015316 47.652484919657...,96.856224
4,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3115149015316 47.652484919657...,96.856224
5,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3124628890345 47.652984815362...,71.509512
6,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3132232503292 47.653393638881...,71.509512
7,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3132232503292 47.653393638881...,129.678151
8,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3132461109814 47.656834269145...,155.162508
9,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3132070900588 47.658314904223...,155.162508


In [251]:
b = r[r['elevation_max']==r['elevation_max'][0]]
B = b.to_json()
B.feature['properties']['elevation_max']

AttributeError: 'str' object has no attribute 'feature'

In [237]:

Seattle_coords = [47.655548, -122.303200]
# f2 = folium.Figure(height = 400)
m2 = folium.Map(location = Seattle_coords, zoom_start = 12)

max_elev = max(r['elevation_max'])
linear = cm.linear.RdBu_06.scale(ele_min.min(), max_elev )
gps_lyr = folium.GeoJson(B, style_function = lambda feature: {
                             'color': linear(feature['properties']['elevation_max']),
                             'weight': 5})

gps_lyr.add_child

m2.add_child(linear)
m2.add_child(gps_lyr)
# m2.add_to(f2)

# plot using points and linestring. If points<x, use points to draw.

In [494]:
Seattle_coords = [47.655548, -122.303200]
# f2 = folium.Figure(height = 400)
m2 = folium.Map(location = Seattle_coords, zoom_start = 12)
max_elev = max(r['elevation_max'])
linear = cm.linear.RdBu_06.scale(ele_min.min(), max_elev )
points = linestringGDF.to_json()
for i in range(len(r)):
    b = r[r['elevation_max']==r['elevation_max'][i]]
    onelinestring = b.to_json()
    if len(geoms[0]['coordinates'][i])<2:

        gps_lyr = folium.GeoJson(points, style_function = lambda feature: {
            'color': linear(feature['properties']['grade']), 'weight': 5})
        gps_lyr.add_child
        m2.add_child(linear)
        m2.add_child(gps_lyr)
        
    else:
        gps_lyr = folium.GeoJson(onelinestring, style_function = lambda feature: {
            'color': linear(feature['properties']['elevation_max']), 'weight': 5})
        gps_lyr.add_child
        m2.add_child(linear)
        m2.add_child(gps_lyr)
        # m2.add_to(f2)
m2

In [525]:
last = len(geoms[0]['coordinates'][i])-1

geoms[0]['coordinates'][i][last]

(-122.30683129285894, 47.650560143135365)

In [531]:
linestringGDF

,id,grade,geometry
0,0,0,LINESTRING (-122.3075605352953 47.650532607492...
0,1,1,LINESTRING (-122.3074727318998 47.650570213082...
0,2,2,LINESTRING (-122.3072258330829 47.650592602839...
0,3,3,LINESTRING (-122.3068467314136 47.650560765602...
0,4,4,LINESTRING (-122.3068312928589 47.650560143135...
0,5,5,LINESTRING (-122.3065875648685 47.650551232044...
0,6,6,LINESTRING (-122.3063251050699 47.650554352899...
0,7,7,LINESTRING (-122.3061037630416 47.650562150965...
0,8,8,LINESTRING (-122.3058318374172 47.650589248717...
0,9,9,LINESTRING (-122.3056157336896 47.650624398824...


# one way route

In [ ]:
six = '../data/six_routes.shp'
data = gpd.read_file(fp)
dat = data[data['IN_SERVICE']=='Y']
da = dat[data['ROUTE_NUM'] == 45]
da